In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_star/1000shot' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.01
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'star/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 1000] LR: 0.100000
Train | 10/10 | Loss:1.2365 | MainLoss:0.9881 | SPLoss:2.3269 | CLSLoss:1.5727 | top1:49.1000 | AUROC:0.0000
Test | 153/10 | Loss:0.9478 | MainLoss:0.6933 | SPLoss:2.3949 | CLSLoss:1.5036 | top1:50.0098 | AUROC:0.4951
Test | 161/10 | Loss:0.9393 | MainLoss:0.6848 | SPLoss:2.3949 | CLSLoss:1.5036 | top1:83.9034 | AUROC:0.9824

Epoch: [2 | 1000] LR: 0.130000
Train | 10/10 | Loss:0.9190 | MainLoss:0.6933 | SPLoss:2.1088 | CLSLoss:1.4840 | top1:50.6000 | AUROC:0.0000
Test | 153/10 | Loss:0.8870 | MainLoss:0.6932 | SPLoss:1.7923 | CLSLoss:1.4602 | top1:50.0262 | AUROC:0.5006
Test | 161/10 | Loss:0.8687 | MainLoss:0.6748 | SPLoss:1.7923 | CLSLoss:1.4602 | top1:87.3551 | AUROC:0.9942

Epoch: [3 | 1000] LR: 0.160000
Train | 10/10 | Loss:0.8613 | MainLoss:0.6935 | SPLoss:1.5346 | CLSLoss:1.4365 | top1:50.0000 | AUROC:0.0000
Test | 153/10 | Loss:0.8343 | MainLoss:0.6952 | SPLoss:1.2501 | CLSLoss:1.4075 | top1:50.0262 | AUROC:0.5107
Test | 161/10 | Loss:0.7850 | Ma

Train | 10/10 | Loss:15.3249 | MainLoss:0.6379 | SPLoss:146.8515 | CLSLoss:0.1885 | top1:66.0000 | AUROC:0.0000
Test | 153/10 | Loss:9.8671 | MainLoss:0.6517 | SPLoss:92.1400 | CLSLoss:0.1424 | top1:66.6219 | AUROC:0.7374
Test | 161/10 | Loss:9.8928 | MainLoss:0.6774 | SPLoss:92.1400 | CLSLoss:0.1424 | top1:58.0062 | AUROC:0.6140

Epoch: [25 | 1000] LR: 0.399833
Train | 10/10 | Loss:7.0993 | MainLoss:0.6719 | SPLoss:64.2577 | CLSLoss:0.1604 | top1:65.6000 | AUROC:0.0000
Test | 153/10 | Loss:4.5175 | MainLoss:0.6474 | SPLoss:38.6847 | CLSLoss:0.1575 | top1:65.9273 | AUROC:0.7213
Test | 161/10 | Loss:4.5537 | MainLoss:0.6837 | SPLoss:38.6847 | CLSLoss:0.1575 | top1:54.2866 | AUROC:0.5863

Epoch: [26 | 1000] LR: 0.399807
Train | 10/10 | Loss:3.4770 | MainLoss:0.6476 | SPLoss:28.2743 | CLSLoss:0.1947 | top1:65.4000 | AUROC:0.0000
Test | 153/10 | Loss:2.6392 | MainLoss:0.6146 | SPLoss:20.2255 | CLSLoss:0.1974 | top1:66.7595 | AUROC:0.7654
Test | 161/10 | Loss:2.6902 | MainLoss:0.6657 | SPLo

Test | 161/10 | Loss:6.1944 | MainLoss:0.6364 | SPLoss:55.5702 | CLSLoss:0.0995 | top1:73.3770 | AUROC:0.9130

Epoch: [47 | 1000] LR: 0.398792
Train | 10/10 | Loss:4.4975 | MainLoss:0.6697 | SPLoss:38.2659 | CLSLoss:0.1205 | top1:59.7500 | AUROC:0.0000
Test | 153/10 | Loss:2.9554 | MainLoss:0.6688 | SPLoss:22.8533 | CLSLoss:0.1331 | top1:58.9515 | AUROC:0.7029
Test | 161/10 | Loss:2.9426 | MainLoss:0.6559 | SPLoss:22.8534 | CLSLoss:0.1331 | top1:59.6293 | AUROC:0.7834

Epoch: [48 | 1000] LR: 0.398722
Train | 10/10 | Loss:3.0870 | MainLoss:0.6560 | SPLoss:24.2968 | CLSLoss:0.1351 | top1:61.9500 | AUROC:0.0000
Test | 153/10 | Loss:4.8969 | MainLoss:0.6493 | SPLoss:42.4668 | CLSLoss:0.0948 | top1:64.7870 | AUROC:0.7232
Test | 161/10 | Loss:4.8960 | MainLoss:0.6484 | SPLoss:42.4668 | CLSLoss:0.0948 | top1:61.4050 | AUROC:0.7780

Epoch: [49 | 1000] LR: 0.398650
Train | 10/10 | Loss:3.5969 | MainLoss:0.6505 | SPLoss:29.4516 | CLSLoss:0.1179 | top1:62.6500 | AUROC:0.0000
Test | 153/10 | Loss:

Test | 153/10 | Loss:4.6230 | MainLoss:0.5273 | SPLoss:40.9278 | CLSLoss:0.3007 | top1:74.1415 | AUROC:0.8216
Test | 161/10 | Loss:4.6469 | MainLoss:0.5511 | SPLoss:40.9277 | CLSLoss:0.3007 | top1:70.7944 | AUROC:0.9122

Epoch: [70 | 1000] LR: 0.039669
Train | 10/10 | Loss:4.3599 | MainLoss:0.4187 | SPLoss:39.3815 | CLSLoss:0.3084 | top1:81.6000 | AUROC:0.0000
Test | 153/10 | Loss:4.2834 | MainLoss:0.5255 | SPLoss:37.5485 | CLSLoss:0.3107 | top1:74.2497 | AUROC:0.8233
Test | 161/10 | Loss:4.3376 | MainLoss:0.5797 | SPLoss:37.5485 | CLSLoss:0.3107 | top1:69.4424 | AUROC:0.9053

Epoch: [71 | 1000] LR: 0.039657
Train | 10/10 | Loss:4.0499 | MainLoss:0.4329 | SPLoss:36.1388 | CLSLoss:0.3145 | top1:79.5500 | AUROC:0.0000
Test | 153/10 | Loss:3.9805 | MainLoss:0.5305 | SPLoss:34.4681 | CLSLoss:0.3129 | top1:73.8139 | AUROC:0.8233
Test | 161/10 | Loss:4.0748 | MainLoss:0.6248 | SPLoss:34.4681 | CLSLoss:0.3129 | top1:66.8536 | AUROC:0.8989

Epoch: [72 | 1000] LR: 0.039646
Train | 10/10 | Loss:

Train | 10/10 | Loss:2.1086 | MainLoss:0.3453 | SPLoss:17.5934 | CLSLoss:0.3877 | top1:85.3500 | AUROC:0.0000
Test | 153/10 | Loss:2.3661 | MainLoss:0.6784 | SPLoss:16.8383 | CLSLoss:0.3854 | top1:70.2359 | AUROC:0.8289
Test | 161/10 | Loss:2.1682 | MainLoss:0.4805 | SPLoss:16.8383 | CLSLoss:0.3854 | top1:76.8879 | AUROC:0.9067

Epoch: [93 | 1000] LR: 0.039356
Train | 10/10 | Loss:2.0160 | MainLoss:0.3873 | SPLoss:16.2488 | CLSLoss:0.3831 | top1:83.4500 | AUROC:0.0000
Test | 153/10 | Loss:2.1169 | MainLoss:0.5581 | SPLoss:15.5507 | CLSLoss:0.3792 | top1:73.2634 | AUROC:0.8373
Test | 161/10 | Loss:2.3382 | MainLoss:0.7793 | SPLoss:15.5507 | CLSLoss:0.3792 | top1:62.9720 | AUROC:0.8786

Epoch: [94 | 1000] LR: 0.039340
Train | 10/10 | Loss:1.8913 | MainLoss:0.3871 | SPLoss:15.0034 | CLSLoss:0.3808 | top1:82.6500 | AUROC:0.0000
Test | 153/10 | Loss:1.9855 | MainLoss:0.5463 | SPLoss:14.3541 | CLSLoss:0.3825 | top1:74.2005 | AUROC:0.8318
Test | 161/10 | Loss:2.1913 | MainLoss:0.7520 | SPLoss

Train | 10/10 | Loss:0.9478 | MainLoss:0.3054 | SPLoss:6.3801 | CLSLoss:0.4451 | top1:87.3000 | AUROC:0.0000
Test | 153/10 | Loss:1.1910 | MainLoss:0.5707 | SPLoss:6.1583 | CLSLoss:0.4486 | top1:75.2785 | AUROC:0.8352
Test | 161/10 | Loss:1.3917 | MainLoss:0.7714 | SPLoss:6.1583 | CLSLoss:0.4486 | top1:68.3925 | AUROC:0.8550

Epoch: [116 | 1000] LR: 0.038942
Train | 10/10 | Loss:0.8747 | MainLoss:0.2721 | SPLoss:5.9819 | CLSLoss:0.4505 | top1:89.5500 | AUROC:0.0000
Test | 153/10 | Loss:1.1737 | MainLoss:0.5927 | SPLoss:5.7640 | CLSLoss:0.4542 | top1:75.1540 | AUROC:0.8362
Test | 161/10 | Loss:1.2523 | MainLoss:0.6714 | SPLoss:5.7640 | CLSLoss:0.4542 | top1:72.6604 | AUROC:0.8868

Epoch: [117 | 1000] LR: 0.038922
Train | 10/10 | Loss:0.8618 | MainLoss:0.2951 | SPLoss:5.6217 | CLSLoss:0.4492 | top1:88.6500 | AUROC:0.0000
Test | 153/10 | Loss:1.1335 | MainLoss:0.5854 | SPLoss:5.4359 | CLSLoss:0.4548 | top1:75.6979 | AUROC:0.8386
Test | 161/10 | Loss:1.2713 | MainLoss:0.7231 | SPLoss:5.435

Test | 161/10 | Loss:1.9202 | MainLoss:0.8955 | SPLoss:10.2021 | CLSLoss:0.4423 | top1:63.9564 | AUROC:0.8239

Epoch: [138 | 1000] LR: 0.038453
Train | 10/10 | Loss:1.2861 | MainLoss:0.2937 | SPLoss:9.8792 | CLSLoss:0.4433 | top1:87.9000 | AUROC:0.0000
Test | 153/10 | Loss:1.6497 | MainLoss:0.6951 | SPLoss:9.5016 | CLSLoss:0.4463 | top1:71.5007 | AUROC:0.8345
Test | 161/10 | Loss:1.5153 | MainLoss:0.5607 | SPLoss:9.5016 | CLSLoss:0.4463 | top1:75.9813 | AUROC:0.8599

Epoch: [139 | 1000] LR: 0.038429
Train | 10/10 | Loss:1.2715 | MainLoss:0.3395 | SPLoss:9.2766 | CLSLoss:0.4375 | top1:86.5000 | AUROC:0.0000
Test | 153/10 | Loss:1.5232 | MainLoss:0.5750 | SPLoss:9.4376 | CLSLoss:0.4391 | top1:75.1999 | AUROC:0.8365
Test | 161/10 | Loss:1.6084 | MainLoss:0.6602 | SPLoss:9.4376 | CLSLoss:0.4391 | top1:72.3801 | AUROC:0.8486

Epoch: [140 | 1000] LR: 0.038405
Train | 10/10 | Loss:1.1647 | MainLoss:0.2412 | SPLoss:9.1905 | CLSLoss:0.4463 | top1:90.4500 | AUROC:0.0000
Test | 153/10 | Loss:1.51

Train | 10/10 | Loss:93.2362 | MainLoss:0.3703 | SPLoss:928.6183 | CLSLoss:0.4119 | top1:82.7000 | AUROC:0.0000
Test | 153/10 | Loss:89.2237 | MainLoss:0.5354 | SPLoss:886.8408 | CLSLoss:0.4166 | top1:75.1180 | AUROC:0.8299
Test | 161/10 | Loss:89.4758 | MainLoss:0.7875 | SPLoss:886.8407 | CLSLoss:0.4166 | top1:65.6075 | AUROC:0.8198

Epoch: [161 | 1000] LR: 0.037849
Train | 10/10 | Loss:85.7426 | MainLoss:0.3251 | SPLoss:854.1321 | CLSLoss:0.4224 | top1:85.8500 | AUROC:0.0000
Test | 153/10 | Loss:82.1393 | MainLoss:0.6094 | SPLoss:815.2567 | CLSLoss:0.4246 | top1:72.7097 | AUROC:0.8290
Test | 161/10 | Loss:82.2469 | MainLoss:0.7171 | SPLoss:815.2569 | CLSLoss:0.4246 | top1:69.1963 | AUROC:0.8467

Epoch: [162 | 1000] LR: 0.037820
Train | 10/10 | Loss:78.8932 | MainLoss:0.3667 | SPLoss:785.2231 | CLSLoss:0.4221 | top1:83.7500 | AUROC:0.0000
Test | 153/10 | Loss:75.4919 | MainLoss:0.5350 | SPLoss:749.5273 | CLSLoss:0.4171 | top1:74.8493 | AUROC:0.8310
Test | 161/10 | Loss:75.7438 | MainL